<a href="https://colab.research.google.com/github/SamuelBM21/IC/blob/main/CNN_LSTM_Inspired_Biometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch                        #PyTorch
import torch.nn as nn               #Camadas da Rede neural
import torch.optim as optim         #Otimizadores
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from torch.optim.lr_scheduler import MultiStepLR
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import seaborn as sns
import copy
import gc

torch.cuda.empty_cache()
gc.collect()

0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class EEGWindowDataset(Dataset):
    def __init__(self, folder_path, subjects, tasks, sampling_points=1920, offset=35, label_map=None, transform=None):
        self.windows = []
        self.labels = []
        self.transform = transform

        for subj in subjects:
            for task in tasks:
                file_name = f"S{subj:03d}{task}.csv"
                file_path = os.path.join(folder_path, f"S{subj:03d}", file_name)

                if not os.path.exists(file_path):
                    print(f"[AVISO] Arquivo não encontrado: {file_path}")
                    continue

                data = np.loadtxt(file_path, delimiter=',')  # (64, total_amostras)
                total_points = data.shape[1]

                for start in range(0, total_points - sampling_points + 1, offset):
                    end = start + sampling_points
                    window = data[:, start:end]
                    if self.transform:
                        window = self.transform(window)
                    self.windows.append(window)
                    self.labels.append(subjects.index(subj))  # índice no vetor de subjects

    def __len__(self):
        return len(self.windows)

    def __getitem__(self, idx):
        x = torch.tensor(self.windows[idx], dtype=torch.float32).permute(1, 0) # Transpose to [1920, 64]
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

In [4]:
class EEGFixedTestDataset(Dataset):
    def __init__(self, folder_path, subjects, tasks=['R01', 'R02'], sampling_points=1920, label_map=None):
        self.windows = []
        self.labels = []

        for subj in subjects:
            for task in tasks:
                file_name = f"S{subj:03d}{task}.csv"
                file_path = os.path.join(folder_path, f"S{subj:03d}", file_name)

                if not os.path.exists(file_path):
                    print(f"[AVISO] Arquivo não encontrado: {file_path}")
                    continue

                data = np.loadtxt(file_path, delimiter=',')  # shape: (64, total_amostras)
                total_points = data.shape[1]

                step = (total_points - sampling_points) // 4
                for i in range(5):
                    start = i * step
                    end = start + sampling_points
                    window = data[:, start:end]
                    self.windows.append(window)
                    self.labels.append(subjects.index(subj))

    def __len__(self):
        return len(self.windows)

    def __getitem__(self, idx):
        x = torch.tensor(self.windows[idx], dtype=torch.float32).permute(1, 0) # Transpose to [1920, 64]
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

In [ ]:
batch_size = 16
learning_rate = 0.01
epochs = 40

# Todos os sujeitos (1-109)
all_subjects = list(range(1, 110))
num_classes = len(all_subjects)  # 109 classes
subject_names = [f"Subject {i}" for i in all_subjects]

# Mapeamento de sujeito para label (0-108)
subject_to_label = {subj: idx for idx, subj in enumerate(all_subjects)}

#Carrega todo o dataset
full_dataset = EEGWindowDataset(
    folder_path='/content/drive/MyDrive/EEGDataset/Dataset_CSV',
    subjects=all_subjects,
    tasks=['R01'],
    sampling_points=1920,
    offset=35,
    label_map=subject_to_label
)

#Extrai todos os índices
indices = list(range(len(full_dataset)))
labels = [full_dataset.labels[i] for i in indices]

# Faz split estratificado (10% para validação)
train_idx, val_idx = train_test_split(
    indices,
    test_size=0.1,
    stratify=labels,
    random_state=42
)


train_dataset = Subset(full_dataset, train_idx)
val_dataset = Subset(full_dataset, val_idx)

test_dataset = EEGFixedTestDataset(
    folder_path='/content/drive/MyDrive/EEGDataset/Dataset_CSV',
    subjects=all_subjects,  # TODOS os sujeitos
    tasks=['R02'],  # OLHOS FECHADOS (teste)
    sampling_points=1920,
    label_map=subject_to_label
)


# Nomes das classes para relatórios (109 sujeitos)
class_names = [f"Subject {i}" for i in all_subjects]
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class EEG_BRNN_LSTM(nn.Module):
    def __init__(self, input_channels=64, time_steps=1920, hidden_size=128, num_layers=3, num_classes=109):
        super().__init__()

        # CNN para extração espacial por canal
        self.cnn = nn.Sequential(

            #Conv1
            nn.Conv1d(128, 96, kernel_size=11),
            nn.ReLU(),
            nn.BatchNorm1d(96),
            nn.MaxPool1d(kernel_size=4),

            #Conv2
            nn.Conv1d(96, 128, kernel_size=9),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.MaxPool1d(kernel_size=2),

            #Conv3
            nn.Conv1d(128, 256, kernel_size=9), # [B, 256, 227]
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.MaxPool1d(kernel_size=2),        # [B, 256, 113]
        )

        # LSTM bidirecional sobre sequência temporal
        self.lstm = nn.LSTM(
            input_size=input_channels,  # output da CNN
            hidden_size=hidden_size,
            num_layers=num_layers,      # Deveria ser 5
            batch_first=True,
        )

        # Classificador
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 113, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # x: [B, 1920, 64]
        out, _ = self.lstm(x)            # [B, 1920, 128]
        x = out.permute(0, 2, 1)         # [B, 128, 1920]
        x = self.cnn(x)                  # [B, 256, 113]
        x = self.classifier(x)           # [B, 109]
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEG_BRNN_LSTM(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[2, 37], gamma=0.1
)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=100):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience, counter = 10, 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validação
        val_loss, val_f1 = evaluate_model(model, val_loader, criterion, device)

        print(f"Época {epoch+1}/{epochs} | Train Loss: {running_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")

        # Early stopping
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_wts = copy.deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Carregar melhor modelo
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def test_model(model, test_loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcular métricas principais
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"\nOverall Accuracy: {accuracy:.4f}")

    # Matriz de confusão reduzida (apenas erros)
    cm = confusion_matrix(all_labels, all_preds)

    # Calcular acurácia por sujeito
    subject_acc = {}
    for i in range(num_classes):
        idx = np.where(np.array(all_labels) == i)[0]
        if len(idx) > 0:
            subject_acc[i] = (np.array(all_preds)[idx] == i).mean()

    # Plotar acurácia por sujeito
    plt.figure(figsize=(15, 6))
    plt.bar(range(num_classes), [subject_acc.get(i, 0) for i in range(num_classes)])
    plt.xlabel("Subject ID")
    plt.ylabel("Accuracy")
    plt.title("Accuracy per Subject")
    plt.xticks(range(num_classes), class_names, rotation=90)
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()

    # Matriz de confusão para os 10 sujeitos com pior desempenho
    worst_subjects = sorted(subject_acc, key=subject_acc.get)[:10]
    worst_indices = [i for i in range(len(all_labels)) if all_labels[i] in worst_subjects]

    if worst_indices:
        cm_worst = confusion_matrix(
            np.array(all_labels)[worst_indices],
            np.array(all_preds)[worst_indices],
            labels=worst_subjects
        )

        plt.figure(figsize=(12, 10))
        sns.heatmap(cm_worst, annot=True, fmt='d', cmap='Blues',
                    xticklabels=[class_names[i] for i in worst_subjects],
                    yticklabels=[class_names[i] for i in worst_subjects])
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix for Worst 10 Subjects")
        plt.show()

In [ ]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(data_loader)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return avg_loss, f1

In [ ]:
model = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    device,
    epochs
)

In [ ]:
print("\nPerformance on Training Data (Eyes Open):")
test_model(model, train_loader, device, subject_names)

In [ ]:
print("\nPerformance on Test Data (Eyes Closed):")
test_model(model, test_loader, device, subject_names)